In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch import nn
import matplotlib.pyplot as plt
import numpy as np



torch.manual_seed(265)
torch.set_default_dtype(torch.double)

## Load Database CIFAR-10 
We load the database and only keep the airplane and bird subsets

In [ ]:
# Paso 1: Cargar el conjunto de datos CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalización
])

full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

img, label = full_train_dataset[0]  # Obtener la primera imagen y etiqueta

print("Print the first image")

def imshow(img):
    img = img / 2 + 0.5     # desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

imshow(img)


# Paso 2: Analizar el conjunto de datos (opcional)
classes = full_train_dataset.classes
print("Classes in CIFAR-10:", classes)

# Paso 3: Preprocesamiento ya incluido en el transform al llamar a datasets

# Paso 4: Dividir el conjunto de entrenamiento en entrenamiento y validación
train_size = int(0.8 * len(full_train_dataset))
validation_size = len(full_train_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_train_dataset, [train_size, validation_size])

# Paso 5: Filtrar por etiquetas "bird" y "plane"
def filter_by_label(dataset, classes_to_keep=['airplane', 'bird']):
    targets = dataset.dataset.targets if isinstance(dataset, Subset) else dataset.targets
    mask = [target in classes_to_keep for target in targets]
    indices = [i for i, m in enumerate(mask) if m]
    return Subset(dataset, indices)

# Filtrando los conjuntos de datos
train_dataset = filter_by_label(train_dataset, [0, 2]) # 0 es 'airplane', 2 es 'bird' en CIFAR-10
validation_dataset = filter_by_label(validation_dataset, [0, 2])
test_dataset = filter_by_label(test_dataset, [0, 2])

# Cargadores de datos para iterar sobre los conjuntos
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")


## MyMLP class

In [ ]:
class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        # Definir las capas del MLP
        self.fc1 = nn.Linear(3072, 512)  # Capa de entrada a la primera capa oculta
        self.fc2 = nn.Linear(512, 128)   # Primera capa oculta a la segunda capa oculta
        self.fc3 = nn.Linear(128, 32)    # Segunda capa oculta a la tercera capa oculta
        self.fc4 = nn.Linear(32, 2)      # Tercera capa oculta a la capa de salida
        # Función de activación ReLU
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Aplanar la entrada para que tenga dimensiones (batch_size, 3072)
        x = x.view(-1, 3072)
        # Pasar por las capas con activación ReLU para las capas ocultas
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        # La capa de salida no lleva activación porque se usará con CrossEntropyLoss
        x = self.fc4(x)
        return x


## Train Function

In [ ]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    model.train()  # Poner el modelo en modo de entrenamiento
    for epoch in range(n_epochs):
        total_loss = 0
        for img, labels in train_loader:
            # Change imgs.to(device=device) to imgs.to(device=device, dtype=torch.double) in your training functions and when computing accuracies in order to convert your images to the right datatype.
            img = img.to(dtype=torch.double)
            optimizer.zero_grad()  # Limpia los gradientes de todos los parámetros optimizados
            outputs = model(img)  # Calcula la predicción del modelo
            loss = loss_fn(outputs, labels)  # Calcula la pérdida
            loss.backward()  # Realiza la retropropagación para calcular los gradientes
            optimizer.step()  # Actualiza los parámetros del modelo

            total_loss += loss.item()  # Acumula la pérdida (opcional) 

        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {total_loss/len(train_loader)}')
